# Join sector timeseries into one file

When running Timeseries_sectors.py the timeseries for each sector is saved as a separate file.
Here we create a new dataset with each sector as a variable, and save as one file.

In [1]:
import os
import glob
import xarray as xr

from acrg.name import name

In [8]:
site = 'WAO'
year = '2015'

### Get the separate timeseries files

In [9]:
ts_path = os.path.join('/user', 'work', 'vf20487', 'Timeseries', 'o2_co2')
ts_files = glob.glob(os.path.join(ts_path, f'{site}_ff-ukghg-*_timeseries_{year}.nc'))
ts_files = {ts_file.split('ukghg-')[1].split('_time')[0]: ts_file
            for ts_file in ts_files}
ts_files

{'offshore': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-offshore_timeseries_2015.nc',
 'energyprod': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-energyprod_timeseries_2015.nc',
 'othertrans': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-othertrans_timeseries_2015.nc',
 'indproc': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-indproc_timeseries_2015.nc',
 'indcom': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-indcom_timeseries_2015.nc',
 'domcom': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-domcom_timeseries_2015.nc',
 'roadtrans': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-roadtrans_timeseries_2015.nc'}

### Join the data

Open the files to get the timeseries for each sector, rename to add the sector into the variable name, and combine into one dataset

In [10]:
ts_data = {sector: name.open_ds(ts_file) for sector, ts_file in ts_files.items()}
ts_data = {sector: ts.rename({'o2': f'o2-{sector}',
                              'co2': f'co2-{sector}'})
           for sector, ts in ts_data.items()}
ts_data_all = xr.merge(ts_data.values())

In [11]:
ts_data_all

<xarray.Dataset>
Dimensions:         (time: 8760)
Coordinates:
  * time            (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
Data variables: (12/14)
    o2-offshore     (time) float32 -0.003238 -0.003306 ... -0.0009775 -0.001129
    co2-offshore    (time) float32 0.002178 0.002224 ... 0.0006643 0.0007675
    o2-energyprod   (time) float32 -0.7337 -0.6964 -0.6635 ... -0.6745 -0.7552
    co2-energyprod  (time) float32 0.5098 0.4836 0.4605 ... 0.2124 0.4598 0.5148
    o2-othertrans   (time) float32 -0.07027 -0.069 -0.06813 ... -0.02528 -0.0264
    co2-othertrans  (time) float32 0.04881 0.04793 0.04733 ... 0.01756 0.01834
    ...              ...
    o2-indcom       (time) float32 -0.5382 -0.4952 -0.4506 ... -0.4004 -0.5783
    co2-indcom      (time) float32 0.3415 0.3138 0.2851 ... 0.1965 0.2503 0.3615
    o2-domcom       (time) float32 -2.454 -2.423 -2.319 ... -1.257 -1.078 -1.163
    co2-domcom      (time) float32 1.308 1.291 1.235 ... 0.6678 0.5727 0.6179
    o2-roadtrans    (time) float32 -1.571 -1.238 -0.9652 ... -1.188 -1.149
    co2-roadtrans   (time) float32 1.09 0.8587 0.6697 ... 0.8657 0.8246 0.7975

### Save

In [12]:
filename = os.path.join(ts_path, f'{site}_ff-ukghg-sectors_timeseries_{year}.nc')
print(f'Saving to {filename}')
ts_data_all.to_netcdf(filename)

Saving to /user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-sectors_timeseries_2015.nc


### Remove separate files

In [13]:
remove_sep_files = [os.remove(ts_file) for ts_file in ts_files.values()]